In [ ]:
#Comenzamos importando las librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, auc, precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import classification_report

#from pycaret.classification import *

from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
import pickle


In [ ]:
with open('/content/my_pipeline.pkl', 'rb') as f:
    pipeline = pickle.load(f)

In [ ]:
#Leemos el dataset con datos de producción
df = pd.read_csv('/content/df_final_cl_v5.csv')
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,meal,market_segment,distribution_channel,is_repeated_guest,...,customer_type,adr,required_car_parking_spaces,total_of_special_requests,contains_agent,is_company,is_family,total_people,total_nights,country_grouped
0,Resort Hotel,0,342,2015,7,1,BB,Direct,Direct,0,...,Transient,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,PRT
1,Resort Hotel,0,737,2015,7,1,BB,Direct,Direct,0,...,Transient,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,PRT
2,Resort Hotel,0,7,2015,7,1,BB,Direct,Direct,0,...,Transient,75.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,GBR
3,Resort Hotel,0,13,2015,7,1,BB,Corporate,Corporate,0,...,Transient,75.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,GBR
4,Resort Hotel,0,14,2015,7,1,BB,Online TA,TA/TO,0,...,Transient,98.0,0.0,1.0,1.0,0.0,0.0,2.0,2.0,GBR


In [ ]:
# 'y_prob' contiene las predicciones probabilísticas de tu modelo
y_prob = pipeline.predict_proba(df)

# Crear un nuevo array con las predicciones ajustadas
y_pred = (y_prob > 0.4).astype(int)

print(y_pred)

[[1 1]
 [1 1]
 [1 0]
 ...
 [0 1]
 [1 1]
 [1 0]]


In [ ]:
y_prob

array([[0.49139535, 0.50860465],
       [0.49139535, 0.50860465],
       [0.8910014 , 0.10899859],
       ...,
       [0.46470416, 0.53529584],
       [0.46653885, 0.53346115],
       [0.39349687, 0.6065031 ]], dtype=float32)

In [ ]:
segundos_elementos = y_prob[:, 1]
print(segundos_elementos)

[0.50860465 0.50860465 0.10899859 ... 0.6101543  0.5254268  0.11610242]


In [ ]:
resultado = (segundos_elementos > 0.4).astype(int)
print (resultado)

[1 1 0 ... 1 1 0]


In [ ]:
y_pred = pipeline.predict(df)

In [ ]:
y_pred

array([1, 1, 0, ..., 1, 1, 1])

In [ ]:
df['hotel'].value_counts()

hotel
City Hotel      53266
Resort Hotel    33947
Name: count, dtype: int64

Dataset con reservas por mes y ver cuales se van a cancelar

In [ ]:
columnas_deseadas = ['hotel', 'arrival_date_year', 'arrival_date_month', 'arrival_date_day_of_month']
df_entregar = df[columnas_deseadas].copy()
df_entregar['cancela'] = resultado

In [ ]:
df_entregar

,hotel,arrival_date_year,arrival_date_month,arrival_date_day_of_month,cancela
0,Resort Hotel,2015,7,1,1
1,Resort Hotel,2015,7,1,1
2,Resort Hotel,2015,7,1,0
3,Resort Hotel,2015,7,1,0
4,Resort Hotel,2015,7,1,1
...,...,...,...,...,...
87208,City Hotel,2017,8,30,0
87209,City Hotel,2017,8,31,1
87210,City Hotel,2017,8,31,1
87211,City Hotel,2017,8,31,1


In [ ]:
# Filtrar el DataFrame para el año 2017
df_2017 = df_entregar[df_entregar['arrival_date_year'] == 2016]

# Agrupar por mes, día y hotel, y calcular el porcentaje de cancelaciones
cancelaciones_por_mes_dia = df_2017.groupby(['hotel', 'arrival_date_month'])['cancela'].mean() * 100

# Resetear el índice para convertir los nombres de hoteles en una columna
cancelaciones_por_mes_dia = cancelaciones_por_mes_dia.reset_index()

print(cancelaciones_por_mes_dia)

           hotel  arrival_date_month    cancela
0     City Hotel                   1  54.854369
1     City Hotel                   2  58.912387
2     City Hotel                   3  61.489632
3     City Hotel                   4  68.451640
4     City Hotel                   5  61.393758
5     City Hotel                   6  65.337423
6     City Hotel                   7  65.536723
7     City Hotel                   8  73.146915
8     City Hotel                   9  84.316038
9     City Hotel                  10  87.139690
10    City Hotel                  11  82.857143
11    City Hotel                  12  87.822878
12  Resort Hotel                   1  27.383863
13  Resort Hotel                   2  29.720280
14  Resort Hotel                   3  40.601504
15  Resort Hotel                   4  39.056743
16  Resort Hotel                   5  39.596381
17  Resort Hotel                   6  43.425325
18  Resort Hotel                   7  54.398827
19  Resort Hotel                   8  61

In [ ]:
df.to_excel('resultado_cancelaciones.xlsx', index=False)
